In [1]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [2]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
        #print(grades)
except IOError:
        print("File not accessible")

Enter a class to grade (i.e. class1 for class1)
Successfully opened class3.txt


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [3]:
def valueCheck(x):
    value = x.split(",")
    length = len(value)
    return length
def has26Values(x):
    if x == 26:
        return True
    else:
        return False
for grade in grades: 
            value = grade.split(",")
            length = len(value)
            if length != 26:
                incorrect_id = value[0]
                print("Invalid line of data:", (incorrect_id) + " does not contain exactly 26 values:")
                print(grade)    
values_count = list(map(valueCheck,grades))
values_check = list(map(has26Values,values_count))
print("Total valid lines of data:", values_check.count(True))
print("Total invalid lines of data:", values_check.count(False))


Total valid lines of data: 100
Total invalid lines of data: 0


Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.compare.html
https://linuxhint.com/pandas-compare-two-dataframes-row-by-row/
pandas.drop 
https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bool.html 

--------------------
How to get the incorrect answer to a new dataframe ( make a new dataframe with the quantity of answering incorrectly, score

In [28]:
answer_keys = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
answer_key = answer_keys.split(",")
answer_key_table = pd.DataFrame(answer_key)
#print(answer_key_table)
for grade in grades: 
        value = grade.split(",")
        answer_table = pd.DataFrame(value)
        answer_table = answer_table.iloc[1: , :]
        answer_table = answer_table.reset_index(drop=True)
        cVal = answer_table.iloc[24][0]
        cValSplit = cVal.split("\n")
        c = cValSplit[0]
        answer_table = answer_table.replace([cVal],c)
        #print(answer_table)
        comparison = answer_key_table.compare(answer_table,align_axis=0,keep_shape=True)
        comparison_table = pd.DataFrame(comparison)
        gradeCheck = comparison_table.value_counts(dropna=False)
        gradeCheck.to_frame()
        score  = gradeCheck[0]
        correctAnswer = score*2
        print(comparison_table)        

            0
0  self   NaN
   other  NaN
1  self     A
   other     
2  self   NaN
   other  NaN
3  self   NaN
   other  NaN
4  self   NaN
   other  NaN
5  self   NaN
   other  NaN
6  self   NaN
   other  NaN
7  self   NaN
   other  NaN
8  self   NaN
   other  NaN
9  self   NaN
   other  NaN
10 self   NaN
   other  NaN
11 self   NaN
   other  NaN
12 self   NaN
   other  NaN
13 self   NaN
   other  NaN
14 self     A
   other    D
15 self   NaN
   other  NaN
16 self   NaN
   other  NaN
17 self   NaN
   other  NaN
18 self   NaN
   other  NaN
19 self     C
   other     
20 self     A
   other     
21 self   NaN
   other  NaN
22 self   NaN
   other  NaN
23 self   NaN
   other  NaN
24 self   NaN
   other  NaN
            0
0  self   NaN
   other  NaN
1  self     A
   other     
2  self   NaN
   other  NaN
3  self     D
   other    C
4  self   NaN
   other  NaN
5  self   NaN
   other  NaN
6  self   NaN
   other  NaN
7  self     A
   other    D
8  self   NaN
   other  NaN
9  self   NaN
   oth

Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'